# Import Stuff

In [27]:
# Cell 1: Import libraries and setup
import os
import json
import logging
from typing import Dict, List, Any, TypedDict
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# LangChain imports
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.agents import create_react_agent
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# LangGraph imports
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

# Tavily search tool from LangChain Community
from langchain_tavily import TavilySearch

# Load environment variables
load_dotenv()

# Configure logging
def setup_logging(level: str = "INFO", log_file: str = None):
    """Setup logging configuration"""
    
    # Create custom formatter
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    # Setup root logger
    root_logger = logging.getLogger()
    root_logger.setLevel(getattr(logging, level.upper()))
    
    # Clear existing handlers
    root_logger.handlers.clear()
    
    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    root_logger.addHandler(console_handler)
    
    # File handler (optional)
    if log_file:
        file_handler = logging.FileHandler(log_file, encoding='utf-8')
        file_handler.setFormatter(formatter)
        root_logger.addHandler(file_handler)
    
    # Configure LangChain/LangGraph specific loggers
    langchain_logger = logging.getLogger("langchain")
    langchain_logger.setLevel(getattr(logging, level.upper()))
    
    langgraph_logger = logging.getLogger("langgraph")
    langgraph_logger.setLevel(getattr(logging, level.upper()))
    
    # Create application logger
    app_logger = logging.getLogger("research_workflow")
    app_logger.setLevel(getattr(logging, level.upper()))
    
    return app_logger

# Setup default logging
logger = setup_logging("INFO")
logger.info("\N{WHITE HEAVY CHECK MARK} All imports and logging setup successful!")

print("\N{WHITE HEAVY CHECK MARK} All imports successful!")

2025-06-18 15:24:52 - research_workflow - INFO - ✅ All imports and logging setup successful!


✅ All imports successful!


# Config

In [28]:
# Cell 3: Configuration and LLM Setup
class Config:
    def __init__(self):
        # API Keys - set these in your .env file or environment
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        self.anthropic_api_key = os.getenv("ANTHROPIC_API_KEY") 
        self.tavily_api_key = os.getenv("TAVILY_API_KEY")
        
        # Default LLM settings
        self.default_model = "gpt-4o-mini"  # or "claude-3-haiku-20240307"
        self.default_temperature = 0.1
        
        # Validate API keys
        if not self.tavily_api_key:
            print("\N{WARNING SIGN}  Warning: TAVILY_API_KEY not found. Web search will not work.")
        if not self.openai_api_key and not self.anthropic_api_key:
            print("\N{WARNING SIGN}  Warning: No LLM API keys found. Please set OPENAI_API_KEY or ANTHROPIC_API_KEY")

def get_llm(model_name: str = None, temperature: float = None):
    """Get configured LLM instance"""
    config = Config()
    model = model_name if model_name else config.default_model
    temp = temperature if temperature is not None else config.default_temperature
    
    if model.startswith("gpt"):
        if not config.openai_api_key:
            raise ValueError("OpenAI API key required for GPT models")
        return ChatOpenAI(
            model=model,
            temperature=temp,
            api_key=config.openai_api_key
        )
    elif model.startswith("claude"):
        if not config.anthropic_api_key:
            raise ValueError("Anthropic API key required for Claude models")
        return ChatAnthropic(
            model=model,
            temperature=temp,
            api_key=config.anthropic_api_key
        )
    else:
        raise ValueError(f"Unsupported model: {model}")

# Test LLM setup
try:
    test_llm = get_llm()
    print(f"\N{WHITE HEAVY CHECK MARK} LLM setup successful: {Config().default_model}")
except Exception as e:
    print(f"\N{CROSS MARK} LLM setup failed: {e}")

✅ LLM setup successful: gpt-4o-mini


# Define Tools

In [29]:
# Use LangChain's built-in Tavily search tool
def get_tavily_tool(max_results: int = 5):
    """Get configured Tavily search tool"""
    return TavilySearch(
        max_results=max_results,
        search_depth="basic",
        include_answer=True,
        include_raw_content=False,
        api_key=Config().tavily_api_key
    )

@tool 
def scrape_website(url: str) -> Dict[str, str]:
    """Scrape content from a website URL"""
    logger = logging.getLogger("research_workflow.webscrape_func")
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        logger.debug(f"\N{GLOBE WITH MERIDIANS} Scraping URL: {url} - Status Code: {response.status_code}")
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer", "header"]):
            script.decompose()
            
        # Get text content
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        # Limit content length
        max_length = 5000
        if len(text) > max_length:
            text = text[:max_length] + "..."
            
        return {
            "url": url,
            "title": soup.title.string if soup.title else "No title",
            "content": text,
            "status": "success"
        }
        
    except Exception as e:
        return {
            "url": url,
            "content": "",
            "status": "error",
            "error": str(e)
        }

# Define Graph State Schema

In [30]:
class ResearchState(TypedDict):
    """State schema for the research workflow"""
    user_query: str
    max_research_steps: int  # New field for step limit
    plan: List[str]
    current_step: int
    research_data: List[Dict[str, Any]]
    final_report: str
    status: str
    messages: List[Any]

# Planner Agent

In [31]:
# Cell 6: Planner Agent
def create_planner_agent(model_name: str = None, temperature: float = None):
    """Create the planner agent that handles both initial planning and revisions"""
    llm = get_llm(model_name, temperature)
    
    planner_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert research assistant. Create or revise a research plan to help gather comprehensive information on the given topic.
        
        For INITIAL planning:
        - Break down research topic into NO MORE THAN {max_steps} sub-topics
        - For each subtopic, add give a brief description of the actions to be taken by a researcher in the format: [subtopic]: [description]
        - Each subtopic should be specific and actionable
        - Subtopics must either be in a logical sequence or MECE (Mutually Exclusive, Collectively Exhaustive)
        - Consider information sources needed for each subtopic

        For REVISIONS:
        - Review the conversation history for user feedback
        - Address user concerns while staying within step limit
        - Maintain plan quality and coherence

        IMPORTANT: Create exactly {max_steps} steps or fewer. Quality over quantity.
        Format as numbered list.

        Query: {query}
        Maximum steps: {max_steps}
        """),
        MessagesPlaceholder(variable_name="messages")
    ])
    
    return planner_prompt | llm

def planner_node(state: ResearchState) -> ResearchState:
    """Planner node - creates or revises research plan based on message history"""
    logger = logging.getLogger("research_workflow.planner")
    
    max_steps = state.get("max_research_steps", 5)
    
    # Check if this is initial planning or revision
    messages = state.get("messages", [])
    is_revision = any(msg.get("role") == "human" and msg.get("content") != "Plan approved" 
                    for msg in messages if isinstance(msg, dict))
    
    if is_revision:
        logger.info(f"\N{CLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCLE ARROWS} Revising research plan based on user feedback")
    else:
        logger.info(f"\N{DIRECT HIT} Creating initial research plan for: '{state['user_query']}'")
    
    logger.info(f"\N{BAR CHART} Maximum research steps allowed: {max_steps}")
    
    try:
        planner = create_planner_agent()
        
        # Convert messages to proper format for the prompt
        formatted_messages = []
        for msg in messages:
            if isinstance(msg, dict):
                if msg.get("role") == "human":
                    formatted_messages.append(HumanMessage(content=msg["content"]))
                elif msg.get("role") == "ai":
                    formatted_messages.append(AIMessage(content=msg["content"]))
            else:
                formatted_messages.append(msg)
        
        # Add initial human message if no messages exist
        if not formatted_messages:
            formatted_messages.append(HumanMessage(content=f"Create a research plan for: {state['user_query']}"))
        
        response = planner.invoke({
            "query": state["user_query"],
            "max_steps": max_steps,
            "messages": formatted_messages
        })
        
        # Parse the plan
        plan_text = response.content
        plan_lines = [line.strip() for line in plan_text.split('\n') 
                     if line.strip() and any(char.isdigit() for char in line[:3])]
        
        plan = []
        for line in plan_lines:
            cleaned = line.split('.', 1)[-1].strip() if '.' in line else line.strip()
            if cleaned:
                plan.append(cleaned)
        
        # Enforce step limit
        if len(plan) > max_steps:
            logger.warning(f"\N{WARNING SIGN} Plan exceeded limit, truncating to {max_steps} steps")
            plan = plan[:max_steps]
        
        action = "Revised" if is_revision else "Generated"
        logger.info(f"\N{CLIPBOARD} {action} {len(plan)} research steps")
        
        return {
            **state,
            "plan": plan,
            "current_step": 0,
            "status": "planning_complete",
            "messages": state.get("messages", []) + [response]
        }
        
    except Exception as e:
        logger.error(f"\N{CROSS MARK} Planning failed: {str(e)}")
        raise

# Human Revision Node

In [32]:
# Cell 6.5: Human Approval Node
def human_approval_node(state: ResearchState) -> ResearchState:
    """Human approval node - seeks user approval for the research plan"""
    logger = logging.getLogger("research_workflow.human_approval")
    
    logger.info("\N{BUST IN SILHOUETTE} Requesting human approval for research plan...")
    
    # Display the plan to the user
    print("\n" + "="*60)
    print("\N{CLIPBOARD} RESEARCH PLAN FOR APPROVAL")
    print("="*60)
    print(f"Query: {state['user_query']}")
    print(f"Max Steps: {state.get('max_research_steps', 5)}")
    print("\nProposed Research Plan:")
    
    for i, step in enumerate(state.get('plan', []), 1):
        print(f"  {i}. {step}")
    
    print("\n" + "="*60)
    print("Please review the plan above.")
    print("- Type [APPROVE] to proceed with research")
    print("- Or provide feedback to modify the plan")
    print("="*60)
    
    # Get user input
    user_input = input("\nYour response: ").strip()
    
    if user_input.upper() == "[APPROVE]":
        logger.info("\N{WHITE HEAVY CHECK MARK} Plan approved by user")
        return {
            **state,
            "status": "plan_approved",
            "messages": state.get("messages", []) + [{"role": "human", "content": "Plan approved"}]
        }
    else:
        logger.info(f"\N{MEMO} User requested plan modification: {user_input}")
        return {
            **state,
            "status": "plan_needs_revision",
            "messages": state.get("messages", []) + [{"role": "human", "content": user_input}]
        }

def route_after_human_approval(state: ResearchState) -> str:
    """Route after human approval"""
    if state.get("status") == "plan_approved":
        return "start_research"
    elif state.get("status") == "plan_needs_revision":
        return "revise_plan"
    else:
        return "human_approval"  # Stay in approval loop

# Research Agent

In [33]:
# Cell 7: Researcher Agent
def create_researcher_agent(model_name: str = None, temperature: float = None):
    """Create the researcher agent with tools"""
    llm = get_llm(model_name, temperature)
    
    # Use LangChain's built-in Tavily tool + custom scraping tool
    tavily_tool = get_tavily_tool(max_results=2)
    tools = [tavily_tool, scrape_website]
    
    # Use the correct 'prompt' parameter
    system_prompt = """You are a research agent that gathers information and provides structured findings with citations.

        Your task for each subtopic:
        1. Use tavily_search to find relevant sources
        2. Use scrape_website on the URLs given
        3. Synthesize findings into a structured format

        Output format:
        ## Key Findings
        - [Finding 1 with citation references]
        - [Finding 2 with citation references]
        - [Finding 3 with citation references] ...

        ## Sources
        [1] Title - Domain (URL)
        [2] Title - Domain (URL)
        [3] Title - Domain (URL) ...

        Guidelines:
        - Reference sources as [1], [2], [3] etc.
        - Focus on factual information and statistics
        - Limit to 400 words total"""
    
    return create_react_agent(llm, tools, prompt=system_prompt)

def researcher_node(state: ResearchState) -> ResearchState:
    """Researcher node using ReAct agent with structured output"""
    logger = logging.getLogger("research_workflow.researcher")
    
    if state["current_step"] >= len(state["plan"]):
        return {**state, "status": "research_complete"}
    
    current_subtopic = state["plan"][state["current_step"]]
    step_num = state["current_step"] + 1
    # Use ReAct agent for this subtopic
    researcher = create_researcher_agent()
    
    research_input = {
        "messages": [HumanMessage(content=f"Research this subtopic with citations: {current_subtopic}")]
    }
    try:
        logger.debug(f"\N{LEFT-POINTING MAGNIFYING GLASS} Researcher is researching subtopic {state['current_step']}")
        result = researcher.invoke(research_input)
        # Extract structured findings
        findings = result["messages"][-1].content
        
        research_data = state.get("research_data", [])
        research_data.append({
            "subtopic": current_subtopic,
            "findings": findings,  # Already includes citations
            "step": step_num
        })
        logger.debug(f"\N{FILE FOLDER} Current research data: {research_data}")
        
        # Overwrite StateGraph with new research data, current step and status
        return {
            **state,
            "research_data": research_data,
            "current_step": step_num,
            "status": "researching"
        }

    except Exception as e:
        logger.error(f"\N{CROSS MARK} Research step {step_num} failed: {str(e)}")
        raise

# Report Writer

In [34]:
# Cell 8: Report Writer Agent
def create_report_writer_agent(model_name: str = None, temperature: float = None):
    """Create the report writer agent"""
    llm = get_llm(model_name, temperature)
    
    report_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert report writer. Your job is to synthesize research findings into a comprehensive, well-structured report.

        Guidelines:
        - Write in clear, professional language
        - Use headings and subheadings for organization
        - Include specific data points and evidence
        - Synthesize information rather than just listing facts
        - Draw meaningful conclusions
        - Use Wikipedia-style citations for sources

        Research Data: {research_data}
        Original Query: {query}"""),
        ("human", "Please create a comprehensive report based on the research findings.")
    ])
    
    return report_prompt | llm

def report_writer_node(state: ResearchState) -> ResearchState:
    """Report writer node - creates final report"""
    logger = logging.getLogger("research_workflow.report_writer")
    
    logger.info("\N{MEMO} Starting report generation...")
    
    try:
        report_writer = create_report_writer_agent()
        
        # Prepare research data for the report
        research_summary = ""
        for i, data in enumerate(state.get("research_data", []), 1):
            logger.debug(f"Processing research step {i}: {data['subtopic']}")
            research_summary += f"\n\n## Research Step {i}: {data['subtopic']}\n{data['findings']}"
        
        logger.debug(f"Prepared research summary with {len(research_summary)} characters")
        
        response = report_writer.invoke({
            "research_data": research_summary,
            "query": state["user_query"]
        })
        
        report_length = len(response.content)
        logger.info(f"\N{WHITE HEAVY CHECK MARK} Report generated successfully ({report_length} characters)")
        
        return {
            **state,
            "final_report": response.content,
            "status": "complete",
            "messages": state.get("messages", []) + [response]
        }
        
    except Exception as e:
        logger.error(f"\N{CROSS MARK} Report generation failed: {str(e)}")
        raise

# Decision Functions (routers)

In [35]:
# Cell 9: Decision Functions
def should_continue_research(state: ResearchState) -> str:
    """Decide whether to continue research or move to report writing"""
    if state["current_step"] >= len(state.get("plan", [])):
        return "write_report"
    else:
        return "continue_research"

# (CURRENTLY UNUSED) Function to start research
def route_after_planning(state: ResearchState) -> str:
    """Route after planning is complete"""
    return "start_research"

def route_after_human_approval(state: ResearchState) -> str:
    """Route after human approval"""
    if state.get("status") == "plan_approved":
        return "start_research"
    elif state.get("status") == "plan_needs_revision":
        return "revise_plan"
    else:
        return "human_approval"  # Stay in approval loop

# Build Graph

In [36]:
# Cell 10: Build the Graph (UPDATED)
def create_research_workflow(model_name: str = None, temperature: float = None):
    """Create the complete research workflow graph"""
    
    # Initialize the graph
    workflow = StateGraph(ResearchState)
    
    # Add nodes
    workflow.add_node("planner", planner_node)
    workflow.add_node("human_approval", human_approval_node)
    workflow.add_node("researcher", researcher_node)
    workflow.add_node("report_writer", report_writer_node)
    
    # Add edges
    workflow.add_edge(START, "planner")
    workflow.add_edge("planner", "human_approval")
    
    # Add conditional edges
    workflow.add_conditional_edges(
        "human_approval",
        route_after_human_approval,
        {
            "start_research": "researcher",
            "revise_plan": "planner"  # Go back to planner for revision
        }
    )
    
    # Add router to check if all subtopics have been researched
    workflow.add_conditional_edges(
        "researcher",
        should_continue_research,
        {
            "continue_research": "researcher",
            "write_report": "report_writer"
        }
    )
    
    workflow.add_edge("report_writer", END)
    
    # Add memory
    memory = MemorySaver()
    
    # Compile the graph
    app = workflow.compile(checkpointer=memory)
    
    return app

# Main Execution Function

In [37]:
# Cell 11: Main Execution Function (Reverted to original, emojis replaced with \N encoding)
def run_research(query: str, model_name: str = None, temperature: float = None, max_research_steps: int = 5, log_level: str = "INFO", log_file: str = None):
    """Run the complete research workflow"""
    
    # Setup logging for this run
    logger = setup_logging(log_level, log_file)
    
    logger.info("\N{ROCKET}" + "="*60)
    logger.info(f"\N{ROCKET} Starting research workflow")
    logger.info(f"\N{CLIPBOARD} Query: {query}")
    logger.info(f"\N{ROBOT FACE} Model: {model_name or Config().default_model}")
    logger.info(f"\N{THERMOMETER}  Temperature: {temperature if temperature is not None else Config().default_temperature}")
    logger.info(f"\N{BAR CHART} Max Research Steps: {max_research_steps}")
    logger.info(f"\N{BAR CHART} Log Level: {log_level}")
    if log_file:
        logger.info(f"\N{PAGE FACING UP} Log File: {log_file}")
    logger.info("\N{ROCKET}" + "="*60)
    
    # Create workflow
    logger.debug("Creating workflow graph...")
    app = create_research_workflow(model_name, temperature)
    
    # Initial state
    initial_state = {
        "user_query": query,
        "max_research_steps": max_research_steps,
        "plan": [],
        "current_step": 0,
        "research_data": [],
        "final_report": "",
        "status": "starting",
        "messages": []
    }
    
    # Configuration for the run
    config = {"configurable": {"thread_id": f"research_{datetime.now().strftime('%Y%m%d_%H%M%S')}"}}
    
    try:
        # Execute the workflow
        logger.info("\N{CLAPPER BOARD} Starting workflow execution...")
        final_state = None
        step_count = 0
        
        for state in app.stream(initial_state, config):
            step_count += 1
            for node_name, node_state in state.items():
                logger.info(f"\N{ROUND PUSHPIN} Step {step_count}: Executing node '{node_name}'")
                
                if node_name == "planner" and node_state.get("plan"):
                    max_steps = node_state.get("max_research_steps", 5)
                    logger.info(f"\N{CLIPBOARD} Research Plan Created ({len(node_state['plan'])} steps, max {max_steps}):")
                    for i, step in enumerate(node_state["plan"], 1):
                        logger.info(f"    {i}. {step}")
                
                elif node_name == "researcher":
                    current_step = node_state.get("current_step", 0)
                    total_steps = len(node_state.get("plan", []))
                    if current_step <= total_steps:
                        progress = f"{current_step}/{total_steps}"
                        logger.info(f"\N{LEFT-POINTING MAGNIFYING GLASS} Research progress: {progress}")
                
                elif node_name == "report_writer":
                    logger.info("\N{MEMO} Final report generated!")
                
                final_state = node_state
        
        # Log completion summary
        if final_state:
            import ipdb; ipdb.set_trace()
            research_steps = len(final_state.get("research_data", []))
            report_length = len(final_state.get("final_report", ""))
            max_steps = final_state.get("max_research_steps", "N/A")
            
            logger.info("\N{WHITE HEAVY CHECK MARK}" + "="*60)
            logger.info("\N{WHITE HEAVY CHECK MARK} Research workflow completed successfully!")
            logger.info(f"\N{BAR CHART} Research steps executed: {research_steps}/{max_steps}")
            logger.info(f"\N{PAGE FACING UP} Final report length: {report_length} characters")
            logger.info(f"\N{STOPWATCH}  Workflow steps: {step_count}")
            logger.info("\N{WHITE HEAVY CHECK MARK}" + "="*60)
        
        return final_state
        
    except Exception as e:
        logger.error("\N{CROSS MARK}" + "="*60)
        logger.error(f"\N{CROSS MARK} Research workflow failed: {str(e)}")
        logger.error("\N{CROSS MARK}" + "="*60)
        raise

# Define Example Usage

In [38]:
# Cell 12: Example Usage and Testing
def display_results(result_state):
    """Display the research results in a formatted way"""
    if not result_state:
        print("\N{CROSS MARK} No results to display")
        return

    print("="*80)
    print("\N{DIRECT HIT} RESEARCH RESULTS")
    print("="*80)

    print(f"\n\N{MEMO} Original Query: {result_state.get('user_query', 'N/A')}")

    if result_state.get('plan'):
        print(f"\n\N{CLIPBOARD} Research Plan:")
        for i, step in enumerate(result_state['plan'], 1):
            print(f"   {i}. {step}")

    if result_state.get('research_data'):
        print(f"\n\N{LEFT-POINTING MAGNIFYING GLASS} Research Steps Completed: {len(result_state['research_data'])}")

    if result_state.get('final_report'):
        print(f"\n\N{PAGE FACING UP} Final Report:")
        print("-" * 40)
        print(result_state['final_report'])

    print("\n" + "="*80)

def get_report(result_state):
    """Get the final report from the result state"""

    return result_state.get('final_report', "No final report generated")

# Test the system with a sample query
print("\N{TEST TUBE} Testing the system...")
print("You can now run research queries using the run_research() function!")
print("\nExample usage:")
print("result = run_research('What are the latest developments in AI safety research?')")
print("display_results(result)")

# Ready to use!
print("\n\N{WHITE HEAVY CHECK MARK} Multi-Agent Web Research System is ready!")
print("\nTo start researching:")
print("result = run_research('your query here', max_research_steps=3)")
print("display_results(result)")

print("\n\N{WRENCH} Research Step Limit Feature:")
print("- Default: 5 steps")
print("- Range: 1-10 steps")
print("- Usage: run_research('query', max_research_steps=3)")

print("\n\N{BAR CHART} Available Parameters:")
print("- query: str (required)")
print("- model_name: str (optional, e.g., 'gpt-4o-mini')")
print("- temperature: float (optional, 0.0-2.0)")
print("- max_research_steps: int (optional, 1-10, default=5)")
print("- log_level: str (optional, 'INFO'/'DEBUG'/'WARNING')")
print("- log_file: str (optional, path to save logs)")

🧪 Testing the system...
You can now run research queries using the run_research() function!

Example usage:
result = run_research('What are the latest developments in AI safety research?')
display_results(result)

✅ Multi-Agent Web Research System is ready!

To start researching:
result = run_research('your query here', max_research_steps=3)
display_results(result)

🔧 Research Step Limit Feature:
- Default: 5 steps
- Range: 1-10 steps
- Usage: run_research('query', max_research_steps=3)

📊 Available Parameters:
- query: str (required)
- model_name: str (optional, e.g., 'gpt-4o-mini')
- temperature: float (optional, 0.0-2.0)
- max_research_steps: int (optional, 1-10, default=5)
- log_level: str (optional, 'INFO'/'DEBUG'/'WARNING')
- log_file: str (optional, path to save logs)


In [39]:
# Example: Uncomment the lines below to run a test
test_result = run_research(
    "Give an overview of hyperspectral cameras, how they work compared to normal cameras, business and industry applications, challenges and limitations.",
    max_research_steps=4,
    log_level="DEBUG",
    log_file="research_debug.log"
)

2025-06-18 15:24:53 - research_workflow - INFO - 🚀============================================================
2025-06-18 15:24:53 - research_workflow - INFO - 🚀 Starting research workflow
2025-06-18 15:24:53 - research_workflow - INFO - 📋 Query: Give an overview of hyperspectral cameras, how they work compared to normal cameras, business and industry applications, challenges and limitations.
2025-06-18 15:24:53 - research_workflow - INFO - 🤖 Model: gpt-4o-mini
2025-06-18 15:24:53 - research_workflow - INFO - 🌡  Temperature: 0.1
2025-06-18 15:24:53 - research_workflow - INFO - 📊 Max Research Steps: 4
2025-06-18 15:24:53 - research_workflow - INFO - 📊 Log Level: DEBUG
2025-06-18 15:24:53 - research_workflow - INFO - 📄 Log File: research_debug.log
2025-06-18 15:24:53 - research_workflow - INFO - 🚀============================================================
2025-06-18 15:24:53 - research_workflow - DEBUG - Creating workflow graph...
2025-06-18 15:24:53 - research_workflow - INFO - 🎬 Start


📋 RESEARCH PLAN FOR APPROVAL
Query: Give an overview of hyperspectral cameras, how they work compared to normal cameras, business and industry applications, challenges and limitations.
Max Steps: 4

Proposed Research Plan:
  1. **Overview of Hyperspectral Cameras**: Research the fundamental principles of hyperspectral imaging, including the technology behind hyperspectral cameras, their spectral resolution, and how they differ from traditional cameras. Sources may include academic journals, technology white papers, and manufacturer specifications.
  2. **Operational Mechanism Compared to Normal Cameras**: Investigate the technical differences in operation between hyperspectral cameras and normal cameras, focusing on aspects such as image capture, data processing, and spectral analysis. Utilize technical manuals, comparison studies, and expert interviews to gather detailed insights.
  3. **Business and Industry Applications**: Explore various sectors where hyperspectral cameras are uti

2025-06-18 15:25:37 - research_workflow.human_approval - INFO - ✅ Plan approved by user
2025-06-18 15:25:37 - research_workflow - INFO - 📍 Step 2: Executing node 'human_approval'
2025-06-18 15:25:37 - research_workflow.researcher - DEBUG - 🔍 Researcher is researching subtopic 0
2025-06-18 15:25:37 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-c15eff9e-b636-4450-9e46-0e4c665b4a86', 'json_data': {'messages': [{'content': 'You are a research agent that gathers information and provides structured findings with citations.\n\n        Your task for each subtopic:\n        1. Use tavily_search to find relevant sources\n        2. Use scrape_website on the URLs given\n        3. Synthesize findings into a structured format\n\n        Output format:\n        ## Key Findings\n        - [Finding 1 with citation references]\n        - [Finding 2 with citation references]\n        - [Finding 3

> c:\users\zicheng liu\appdata\local\temp\ipykernel_19624\1947997507.py(70)run_research()

<class 'dict'>
dict_keys(['user_query', 'max_research_steps', 'plan', 'current_step', 'research_data', 'final_report', 'status', 'messages'])
'Give an overview of hyperspectral cameras, how they work compared to normal cameras, business and industry applications, challenges and limitations.'
4
4
[{'subtopic': '**Overview of Hyperspectral Cameras**: Research the fundamental principles of hyperspectral imaging, including the technology behind hyperspectral cameras, their spectral resolution, and how they differ from traditional cameras. Sources may include academic journals, technology white papers, and manufacturer specifications.', 'findings': '## Key Findings\n- **Fundamental Principles**: Hyperspectral imaging (HSI) captures detailed spectral information across a wide range of wavelengths, typically from ultraviolet to shortwave infrared. Unlike traditional cameras that capture only three color

2025-06-18 15:36:12 - research_workflow - INFO - ✅============================================================
2025-06-18 15:36:12 - research_workflow - INFO - ✅ Research workflow completed successfully!
2025-06-18 15:36:12 - research_workflow - INFO - 📊 Research steps executed: 4/4
2025-06-18 15:36:12 - research_workflow - INFO - 📄 Final report length: 10018 characters
2025-06-18 15:36:12 - research_workflow - INFO - ⏱  Workflow steps: 7
2025-06-18 15:36:12 - research_workflow - INFO - ✅============================================================


In [45]:
test_result.keys()

dict_keys(['user_query', 'max_research_steps', 'plan', 'current_step', 'research_data', 'final_report', 'status', 'messages'])

In [40]:
with open("research_langgraph_seq_result.md", "w") as f:
    f.write(get_report(test_result))

get_report(test_result)

'# Comprehensive Report on Hyperspectral Cameras\n\n## Introduction\nHyperspectral imaging (HSI) represents a significant advancement in imaging technology, enabling the capture of detailed spectral information across a wide range of wavelengths. This report synthesizes research findings on the fundamental principles of hyperspectral cameras, their operational mechanisms compared to traditional cameras, their applications across various industries, and the challenges and limitations associated with their use.\n\n## 1. Overview of Hyperspectral Cameras\n\n### 1.1 Fundamental Principles\nHyperspectral cameras capture detailed spectral information across a wide range of wavelengths, typically from ultraviolet to shortwave infrared. Unlike traditional cameras that capture only three color bands (red, green, blue), hyperspectral cameras record hundreds of narrow spectral bands at each pixel. This capability allows for precise material identification and differentiation based on unique spect